# Objectif

Simplement de mettre en oeuvre les connaissances acquises avec la specialisation Deep Learning sur Coursera.  

On va tenter de determiner a partir de la reponse a la question 2, quelle etait la reponse a la question 1 choisie. 

## imports

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# imports dans l'ordre d'utilisation 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from tensorflow import keras

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Preparation des donnees La transition Ecologique

## chargement des donnees

In [ ]:
nRowsRead = None # specify 'None' if want to read whole file
df = pd.read_csv('../input/granddebat/LA_TRANSITION_ECOLOGIQUE.csv', delimiter=',', nrows = nRowsRead)
df.dataframeName = 'LA_TRANSITION_ECOLOGIQUE.csv'
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')

In [ ]:
df.head(1)

In [ ]:
df.columns.values.tolist()

## Selection des colonnes 
correspondant aux questions 1 & 2

In [ ]:
q1q2_tmp = df.loc[:,['id', "QUXVlc3Rpb246MTYw - Quel est aujourd'hui pour vous le problème concret le plus important dans le domaine de l'environnement ?", 'QUXVlc3Rpb246MTYx - Que faudrait-il faire selon vous pour apporter des réponses à ce problème ?']]
q1q2_tmp.head(1)

In [ ]:
q1q2 = q1q2_tmp.rename(columns={"QUXVlc3Rpb246MTYw - Quel est aujourd'hui pour vous le problème concret le plus important dans le domaine de l'environnement ?": "Q1", 'QUXVlc3Rpb246MTYx - Que faudrait-il faire selon vous pour apporter des réponses à ce problème ?': "Q2"})
q1q2.head(1)

## Selection des lignes
* Conservation des lignes avec une des 4 reponses cochables sur Q1
* Conservation des lignes avec des reponses sur Q2

In [ ]:
q1q2['Q1'].value_counts().head(4)

In [ ]:
q1q2[q1q2['Q1']=='Les dérèglements climatiques (crue, sécheresse)']

In [ ]:
q1q2_lignes_1 = q1q2[q1q2['Q1'].isin(['Les dérèglements climatiques (crue, sécheresse)', 'La biodiversité et la disparition de certaines espèces', "La pollution de l'air", "L'érosion du littoral"])]
q1q2_lignes_1.head(5)

In [ ]:
q1q2_lignes_2 = q1q2_lignes_1[pd.notna(q1q2_lignes_1['Q1']) & pd.notna(q1q2_lignes_1['Q2'])]
q1q2_lignes_2.head(5)

## Encodage de la variable predite

Association d'un chiffre a chaque reponse

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(q1q2_lignes_2['Q1'])
le.classes_

application a la colonne Q1 => creation d'une colonne Q1E pour colonne Q1 Encodee 

In [ ]:
q1q2_lignes_2.loc[:,'Q1E'] = le.transform(q1q2_lignes_2['Q1']).tolist()
q1q2_lignes_2.head(3)

La variable predite doit aussi etre onehot encoded/reshapee en (m,c) avec m le nombre de lignes dans q1q2_lignes_2 et c le nombre de categories.

In [ ]:
# Note: reshape(-1, 1) permet ca, -1 indiquant de conserver le nombre de ligne, cf doc de reshape
# y = q1q2_lignes_2['Q1E'].values.reshape(-1,1)
# ici on fait un one hot encoding
y = to_categorical(q1q2_lignes_2['Q1E'].values)
y, y.shape

In [ ]:
X = q1q2_lignes_2['Q2']
X, X.shape

## Scission en jeux de donnees de test et train

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state = 333)
X_train, X_test, y_train, y_test

# Choix de la métrique

cf https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/  
emojify notebook of week 2 of course 5 Sequence Models  
https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics  

L'entropie croisée est la metrique de base.

# Word embedding

Comment representer les mots utilises dans le formulaire de La Transition Ecologique.
La methode utilisee ici sera d'entrainer une couche d'Embedding de Keras.  

Le texte des reponses doit etre prepare en amont cf https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

Dans un premier temps, pas de pretraitement du genre suppression de mots ou utilisation d'un dictionnaire. 

## Pretraitement pour le word embedding
* tokenization
* conversion en sequence
* padding pour que toutes les sequences aient la meme taille

In [ ]:
max_words = 1001
max_len = 1000
# http://faroit.com/keras-docs/1.2.2/preprocessing/text/
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
# https://keras.io/preprocessing/sequence/
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

A quoi ressemble les sequences_matrix ?

In [ ]:
sequences_matrix

# Modele

Modification du modele emojify_V2 du notebook semaine 2 du cours 5 Sequence Models
melangee avec le notebook Simple LSTM for text classification

## Construction du graphe du modele

In [ ]:
def modelgraph_transition_eco():
    """
    Function creating the Emojify-v2 (nope the transition eco) model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sequences_matrix as the input of the graph, it should be of shape [max_len]
    inputs = Input(name='inputs',shape=[max_len])
    
    # Embedding layer to be learned
    embedding_layer = Embedding(max_words,50,input_length=max_len)(inputs)
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(units = 128, return_sequences = True)(embedding_layer)
    # Add dropout with a probability of 0.5
    X = Dropout(rate=0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(units = 128)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(rate=0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(4)(X)
    # Add a softmax activation
    X = Activation(activation="softmax")(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=inputs, outputs=X)
    
    ### END CODE HERE ###
    
    return model

Appel de la fonction pour creation du modele

In [ ]:
model = modelgraph_transition_eco()
model.summary()

## Definition de la metrique et compilation

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Entrainement du modele

Modifier les valeurs d'epochs et batch_size permet d'obtenir de meilleurs resultats
Par exemple, epoch=1 et batch_size=1024 donne de moins bons resultats que epoch=5 et batch_size=256

In [ ]:
model.fit(sequences_matrix, y_train, epochs = 1, batch_size = 1024, shuffle=True)

## Sauvegarde du modele

Ressource : https://www.tensorflow.org/guide/keras/save_and_serialize

In [ ]:
model.save('/kaggle/working/modele_lstm_classif_gdte.h5')

test consistant a recreer le modele a partir du fichier modele_lstm_classif_gdte

In [ ]:
modele_lstm_classif_gdte = keras.models.load_model('/kaggle/working/modele_lstm_classif_gdte.h5')
modele_lstm_classif_gdte

# Application aux donnees de test

In [ ]:
sequences_test = tok.texts_to_sequences(X_test)
sequences_matrix_test = sequence.pad_sequences(sequences_test,maxlen=max_len)
sequences_matrix_test

In [ ]:
loss, acc = modele_lstm_classif_gdte.evaluate(sequences_matrix_test, y_test)
print()
print("Test accuracy = ", acc)

# Visualisation des resultats

Faisons par exemple une matrice des resultats obtenus pour le test
avec en ligne les vrais réponses et en colonne les réponses prédites...  

In [ ]:
pred_test = modele_lstm_classif_gdte.predict(sequences_matrix_test)
pred_test

Transformation de pred_test et y_test afin d'avoir des array de shape (nb_cas_test,), de valeurs parmi [0,1,2,3] qui correspondent aux reponses de Q1

In [ ]:
pd.crosstab(np.argmax(y_test, axis=1), np.argmax(pred_test, axis=1), rownames=['Actual'], colnames=['Predicted'], margins=True)

Bizarrement, il n'y a aucune prediction 0 (l'erosion du littoral...), qui est tres certainement la reponse minoritaire a Q1 (parmi les cases cochees), voir d'autres notebooks pour s'en assurer.